In [5]:
import requests
import time
import pandas as pd
import ast # to transform str to dic
import praw #for reddit wrapper
import os

In [6]:
# Testing connection

url = 'https://www.reddit.com/hot.json'
header = {'User-agent': 'Bleep blorp bot 0.1'}
res = requests.get(url,headers=header)
the_json = res.json()
print(sorted(the_json.keys()))

the_json['data'].keys()
len(the_json['data']['children'])

ConnectionError: HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: /hot.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f89b992fe20>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
def get_posts(url,interations,header,sleep):
    posts = []
    after = None
    for i in range(interations):
#         print(i)
        if after==None:
            params = {}
        else:
            params = {'after' : after}
        res = requests.get(url,params=params,headers=header)
        if res.status_code == 200:
            the_json = res.json()
            posts.extend(the_json['data']['children'])
            after = the_json['data']['after']
        else:
            print(res.status_code)
            break
        time.sleep(sleep)
    return posts

def create_cols(dataframe):
    dataframe['subreddit'] = dataframe['data'].map(lambda x: x['subreddit'])
    dataframe['title'] = dataframe['data'].map(lambda x: x['title'])
    dataframe['name'] = dataframe['data'].map(lambda x: x['name'])
    dataframe['selftext'] = dataframe['data'].map(lambda x: x['selftext'])
    dataframe['domain'] = dataframe['data'].map(lambda x: x['domain'])
    return dataframe

In [ ]:
# Setting up the Reddit API in python
reddit = praw.Reddit(client_id='mBPz4HF59_0MX5n7TXFGXg',
                     client_secret='L0YrbwjFpyLZs7ia5hTIUyEKuGVrvw',
                     user_agent='User')

# Function for extracting data
def get_posts(subred_name, n, time = 'all'):
    subreddit = reddit.subreddit(subred_name)
    posts_info = [] 
    
    for subm in subreddit.top(limit=n, time_filter=time):
        
        subred_info = []
        subred_info.append(subm.id)  
        subred_info.append(str(subm.author))
        subred_info.append(subm.title)
        subred_info.append(subm.selftext)
        subred_info.append(subm.score)  
        subred_info.append(subm.upvote_ratio)
        subred_info.append(subm.num_comments)
        subred_info.append(subm.subreddit)
        posts_info.append(subred_info)
    
    sorted_info = sorted(posts_info, key=lambda x: x[1], reverse = True)
    posts_df = pd.DataFrame(sorted_info, columns = ['id','author', 'title', 'text', 'score','upvote_ratio' ,'num_comments', 'subreddit'])
    return posts_df

In [ ]:
rant_df = get_posts('rant',100,time="year")
OMC_df = get_posts('offmychest',100,time="year")
TOMC_df = get_posts('trueoffmychest',100,time="year")
self_df = get_posts('self',100,time="year")

In [ ]:
OMC_df['subreddit'] = "offmychest"
TOMC_df['subreddit'] = 'TrueOffMyChest'
rant_df['subreddit'] = 'rant'
self_df['subreddit'] = 'self'

full_df = pd.concat([OMC_df, TOMC_df, rant_df, self_df])

In [ ]:
full_df

In [ ]:
# Load the regular expression library
import re
def cleanUp(df, columnTo, columnFrom):
    # Remove punctuation
    df[columnTo] = df[columnFrom].map(lambda x: re.sub('[,\.!?]', '', x))
    # Convert the titles to lowercase
    df[columnTo] = df[columnFrom].map(lambda x: x.lower())
    return df

In [ ]:
rant_df = cleanUp(rant_df, 'textClean', 'text')
OMC_df = cleanUp(OMC_df, 'textClean', 'text')
TOMC_df = cleanUp(TOMC_df, 'textClean', 'text')
self_df = cleanUp(self_df, 'textClean', 'text')
full_df = cleanUp(full_df, 'textClean', 'text')

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(self_df['textClean'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = OMC_df.textClean.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

In [ ]:
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

In [ ]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = OMC_df.textClean.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['alarm', 'go', 'morning', 'weekday', 'sleepily', 'reach', 'hit', 'snooze', 'button', 'expect', 'feel', 'wife', 'snuggle', 'remember', 'bed', 'alone', 'get', 'turn', 'light', 'make', 'bed', 'never', 'usually', 'still', 'sleep', 'usually', 'make', 'bed', 'today', 'need', 'help', 'kid', 'get', 'ready', 'school', 'door', 'uncommon', 'always', 'help', 'kid', 'thing', 'drive', 'young', 'preschool', 'get', 'married', 'year', 'ago', 'year', 'well', 'last', 'disagree', 'frequently', 'often', 'become', 'frustrated', 'last', 'big', 'fight', 'year', 'ago', 'learn', 'work', 'disagreement', 'mutual', 'love', 'respect', 'half', 'corny', 'metaphor', 'feel', 'corny', 'true', 'start', 'date', 'tell', 'health', 'bad', 'understand', 'serious', 'drive', 'hospital', 'week', 'wedding', 'year', 'ago', 'first', 'time', 'marriage', 'truly', 'come', 'close', 'die', 'year', 'ago', 'almost', 'bleed', 'routine', 'procedure', 'spend', 'several', 'hour', 'waiting', 'room', 'desperate', 'news', 'doctor', 'terrify', 

In [ ]:
!python -m spacy download en_core_web_md 